In [939]:
import pandas as pd
import numpy as np
from row_col import for_two, for_one
from function import *

In [940]:
def possibilidades_4():
  op = [
        [], [], [], [], # Vertical
        [], [],         # Horizontal
      ]

  DICT = {
    "C'D'": "C'D ",
    "C'D ": "C D ",
    "C D ": "C D'",
    "C D'": "C'D'",
  }

  for k, v in DICT.items():
    op[0].append(f"A'B'A'B {k}{v}")

    op[1].append(f"A'B A B {k}{v}")

    op[2].append(f"A B A B'{k}{v}")

    op[3].append(f"A'B'A B'{k}{v}")

  for i, j in zip(for_two(['A', 'B']), for_two(['C', 'D'])):
    # Mapeando na horizontal
    c = "".join( for_two(['C', 'D']))
    op[4].append(f"{i}{c}")

    # Mapeando na Vertical
    c = "".join( for_two(['A', 'B']) )
    op[5].append(f"{j}{c}")

  return op

In [941]:
def possibilidades_2():
  # Esse array tem todas as possibilidades de 2 adjacentes
  op = [
      [],[], [], [], # Vertical
      [],[], [], []  # Horizontal
     ]
  for i,j in zip(for_two(["C", "D"]), for_two(["A", "B"])):
    op[0].append(f"A'B'A'B {i}")
    op[4].append(f"{j}C'D'C'D ")
    
    op[1].append(f"A'B A B {i}")
    op[5].append(f"{j}C'D C D ")

    op[2].append(f"A B A B'{i}")
    op[6].append(f"{j}C D C D'")
    
    op[3].append(f"A'B'A B'{i}")
    op[7].append(f"{j}C'D'C D'")

  return op

In [942]:
def find_expression(args, len_bit):

    if len(args) != 2**len_bit:
        print('Error!')
        print('args must be equal to ', 2**len_bit)
        return    
    
    len_1 = [0,1] * 16
    len_2 = [0,0,1,1] * 4
    len_3 = np.array([ [0] * 4, [1] * 4 ] * 2 ).reshape(1,-1)[0]
    len_4 = np.array([ [0] * 8, [1] * 8 ] * 2 ).reshape(1,-1)[0]

    letter = ["A", "B","C", "D"]
    bits = []
    bits_s = ""
    
    for i in range(2 ** len_bit):
        if len_bit >= 4:
            bits_s += str(len_4[i])
            # print(len_4[i], end="|")
        if len_bit >= 3:
            bits_s += str(len_3[i])
            # print(len_3[i], end="|")
        if len_bit >= 2:
            bits_s += str(len_2[i])
            # print(len_2[i], end="|")
        if len_bit >= 1:
            # print(len_1[i], "|",args[i])
            bits_s += str(len_1[i])
            bits.append(bits_s)
            bits_s = ""
    
    res = []
    for i in range(0,len(args)):
        if args[i] == 1:
            
            for j in range(len(bits[i])):
                if bits[i][j] == "0":
                    res.append(f"{letter[j]}'")
                    continue
                res.append(letter[j])
    
    expr = ""
    aux = 0
    for i in range(0, len(args)):
        if args[i] == 1:
            expr += "(" + "".join(res[aux:aux+len_bit]) + ") + "
            aux += len_bit
        
    return expr[0:-2], letter[0:len_bit], args

In [943]:
def map_k(exp):
  ret = find_expression(exp, len_bit=4)[0]
  # print('Expressão original: ',ret)

  lista = [
           [exp[0], exp[4], exp[12], exp[8]], 
           [exp[1], exp[5], exp[13], exp[9]],
           [exp[3], exp[7], exp[15], exp[11]],
           [exp[2], exp[6], exp[14], exp[10]]
          ]

  x = np.array(lista).T

  data = pd.DataFrame(x, index=for_two(["A","B"]),columns=for_two(["C","D"]))

  list_with_bits = [
                   [f"{exp[0]} - 0000", f"{exp[4]} - 0100", f"{exp[12]} - 1100", f"{exp[8]} - 1000"], 
                   [f"{exp[1]} - 0001", f"{exp[5]} - 0101", f"{exp[13]} - 1101", f"{exp[9]} - 1001"],
                   [f"{exp[3]} - 0011", f"{exp[7]} - 0111", f"{exp[15]} - 1111", f"{exp[11]} - 1011"],
                   [f"{exp[2]} - 0010", f"{exp[6]} - 0110", f"{exp[14]} - 1110", f"{exp[10]} - 1010"]
                  ]

  x = np.array(list_with_bits).T

  data_with_bits = pd.DataFrame(x, index=for_two(["A","B"]),columns=for_two(["C","D"]))

  return data, data_with_bits

In [944]:
def expr(map_with_bits, map_k, s=1):
    expression = []
    exp_list = []
    # print("Dentro da função expr", map_with_bits)
    # print("Dentro da função expr", map_k)

    for i in range(np.count_nonzero(map_k == int(s))):
      expression.append(map_with_bits.loc[ 
                                            [list(map_k.iloc[np.where(map_k == int(s))].index)[i]],
                                            [list(map_k.iloc[np.where(map_k == int(s))].columns)[i]] 
                            ].values[0][0][4:])
      
      exp_list.append(f"{list(map_k.iloc[np.where(map_k == int(s))].index)[i]}{list(map_k.iloc[np.where(map_k == int(s))].columns)[i]}")
    
    # print("Letras: ",exp_list)
    # print(f"Expressão numerica {s}: ", expression)
    return exp_list

def invert(letter):
  letter_inv = []  

  for i in letter:
    aux = ""

    for j in range(0,len(i), 2):
      
      if i[j] == "A":
        aux += f"C{i[j + 1]}"
      elif i[j] == "B":
        aux += f"D{i[j + 1]}"
      elif i[j] == "C":
        aux += f"A{i[j + 1]}"
      else:
        aux += f"B{i[j + 1]}"

    letter_inv.append(aux)

  return letter_inv

In [945]:
lista_fazendo = []

def solve(expression, w=None):
  if w != None:
    exp = [[], [], [], []]
    final = []
    # print("Expression SOLVE",expression) # ["A'B'C D'", "A'B C D'"] 
    
    for i in expression:
      exp[0].append(i[0:2])
      exp[1].append(i[2:4])
      exp[2].append(i[4:6])
      exp[3].append(i[6:8])
    # print(exp)
    for i in range(4):
      if len(set(exp[i])) == 1:
        final.append(list(set(exp[i]))[0])

    lista_fazendo.append("".join(final))

  else:
    exp = [[], [], [], []]
    final = []
    # print("PRINT",expression)
    
    for i in expression:
      exp[0].append(i[0:2])
      exp[1].append(i[2:4])
      exp[2].append(i[4:6])
      exp[3].append(i[6:8])

    for i in range(4):
      if len(set(exp[i])) == 1:
        final.append(list(set(exp[i]))[0])

    print("x = ", "".join(final))


In [959]:
def case_8(map_k, map_with_bits):
  print("HERE - 8")
  
  # Lista com todas as possibilidades para agrupamento de 8
  op = possibilidades_8()

  # Copiar o mapa para fazer alterações
  map_k_copy = map_k.copy()

  dict_with_row_column = {"Ag_8": {"index": [], "column": []}}

  for j in op:
    for i in j:
      try:
        if np.all(map_k.loc[ [i[0:4], i[4:8] ], [ i[8:12], i[12:16], i[16:20], i[20:] ]].values == [[1,1,1,1]]*2):
          # print(map_k.loc[ [i[0:4], i[4:8] ], [ i[8:12], i[12:16], i[16:20], i[20:] ]])
          
          # Copia para pegar a simplificação para o agrupamento
          map_k_copy_0 = map_k_copy.copy()
          map_k_copy_0.loc[ [i[0:4], i[4:8] ], [ i[8:12], i[12:16], i[16:20], i[20:] ]] = [[2,2,2,2]]*2
          solve(expr(map_with_bits, map_k_copy_0, "2"), "o")
          
          # Tirando o agrupamento, ainda existe 1
          if np.count_nonzero(map_k_copy_0 == 1) >= 0:
            print("AG de 8 try 1")
            # print(map_k_copy)

            dict_with_row_column["Ag_8"]["index"].append([ i[0:4], i[4:8] ])
            dict_with_row_column["Ag_8"]["column"].append([ i[8:12], i[12:16], i[16:20], i[20:] ])
          
            map_k_copy.loc[ [ i[0:4], i[4:8] ], [ i[8:12], i[12:16], i[16:20], i[20:] ]] = [[0,0,0,0]]*2
          
      except:
        if np.all(map_k.loc[ [ i[8:12], i[12:16], i[16:20], i[20:] ] ,[ i[0:4], i[4:8] ] ].values == [[1,1]]*4):
          # print(map_k.loc[ [ i[8:12], i[12:16], i[16:20], i[20:] ] ,[ i[0:4], i[4:8] ] ])

          # Copia para pegar a simplificação para o agrupamento
          map_k_copy_0 = map_k_copy.copy()
          map_k_copy_0.loc[ [ i[8:12], i[12:16], i[16:20], i[20:] ] ,[ i[0:4], i[4:8] ] ] = [[2,2]]*4
          solve(expr(map_with_bits, map_k_copy_0, "2"), "o")
          
          # Tirando o agrupamento, ainda existe 1
          if np.count_nonzero(map_k_copy_0 == 1) >= 0:
            print("AG de 8 try 2")
            # print(map_k_copy)

            dict_with_row_column["Ag_8"]["index"].append([ i[8:12], i[12:16], i[16:20], i[20:] ])
            dict_with_row_column["Ag_8"]["column"].append([ i[0:4], i[4:8] ])
          
            map_k_copy.loc[ [ i[8:12], i[12:16], i[16:20], i[20:] ] ,[ i[0:4], i[4:8] ] ] = [[0,0]]*4
  
  if np.count_nonzero(map_k_copy == 1) == 0:
    print('Resolvi no case 8')
    
    if lista_fazendo != []:
      print(" + ".join(lista_fazendo))
      lista_fazendo.clear()
      return

    # print(list_with_bits)
    
    solve(expr(map_with_bits, map_k,))

    return

  else:
    print("Passando pelo o 8 >>> ", lista_fazendo)

    # print("DICT IN AG 8 - ",dict_with_row_column["Ag_8"])

    # Tudo indo certo, falta so criar uma logica
    # Para pegar o que falta aqui e juntar com o de 4 2 1
    if len(dict_with_row_column["Ag_8"]['index']) > 0:
      teste02(map_k_copy, map_k_copy_0, map_with_bits, dict_with_row_column)
      
    else:
      case_4(map_k, map_with_bits, dict_with_row_column)

In [960]:
def teste(map_k_copy, map_k_copy_0):
  print("*********** DENTRO DE TESTE *************")
  print(map_k_copy)
  print(list(map_k_copy.iloc[ np.where(map_k_copy == 1) ].index))
  print(list(map_k_copy.iloc[ np.where(map_k_copy == 1) ].columns))
  print(map_k_copy_0)

def teste02(map_k_copy, map_k_copy_0, map_with_bits, dict_with_row_column):
  # map_test = map_k_copy_0.copy()
  # print(map_k_copy)

  if np.count_nonzero(map_k_copy == 1) >= 2:
    print('EXISTE POSSIBILIDADES DE TER AG DE 4')
    print(map_k_copy)

    for j in possibilidades_4():
      for i in j:
        try:
    
            if np.any(map_k_copy_0.loc[ [i[0:4], i[4:8] ], [ i[8:12], i[12:] ]].values == 1) and np.all(map_k_copy_0.loc[ [i[0:4], i[4:8] ], [ i[8:12], i[12:] ]].values != 0):
              
              # Alterando em um df diferente
              map_k_copy.loc[ [i[0:4], i[4:8] ], [ i[8:12], i[12:] ]] = [[1,1], [1,1]]

              
        except:
          try:
            if np.any(map_k_copy_0.loc[ [i[0:4] ], [ i[4:8], i[8:12], i[12:16], i[16:] ]].values == 1) and np.all(map_k_copy_0.loc[ [i[0:4] ], [ i[4:8], i[8:12], i[12:16], i[16:] ]].values != 0):
              
              
              map_k_copy.loc[ [i[0:4] ], [ i[4:8], i[8:12], i[12:16], i[16:] ]] = [[1,1,1,1]] 
            
          except:
            if np.any(map_k_copy_0.loc[ [ i[4:8], i[8:12], i[12:16], i[16:] ], [i[0:4] ]].values == 1) and np.all(map_k_copy_0.loc[ [ i[4:8], i[8:12], i[12:16], i[16:] ], [i[0:4] ]].values != 0):
              print("AQUI 3")
              map_k_copy.loc[ [ i[4:8], i[8:12], i[12:16], i[16:] ], [i[0:4] ]] = [[1], [1],[1], [1]]
    
    
    if np.count_nonzero(map_k_copy == 1) >= 4:
      
      # Se resolveu todos os casos (PARE!)
      ret = case_4(map_k_copy, map_with_bits, dict_with_row_column, veio=True)

      if np.count_nonzero(ret == 1) == 0:
        return
      else:
        map_k_copy_0 = ret

        if np.count_nonzero(map_k_copy_0 == 1) == 1:
          case_1(map_k_copy_0, map_with_bits)
          return      

  if np.count_nonzero(map_k_copy == 1) >= 1:
    print('EXISTE POSSIBILIDADES DE TER AG DE 2')
    print(map_k_copy_0)

    for j in possibilidades_2():
      for i in j:
        try:
    
            if np.any(map_k_copy_0.loc[ [i[0:4], i[4:8] ], [i[8:]]].values == 1) and np.all(map_k_copy_0.loc[ [i[0:4], i[4:8] ], [i[8:]]].values != 0):
              
              print('AQUI 1')
              # Alterando em um df diferente
              map_k_copy.loc[ [i[0:4], i[4:8] ], [i[8:]]] = [[1], [1]]

        except:
          
            if np.any(map_k_copy_0.loc[ [i[0:4] ], [ i[4:8], i[8:]]].values == 1) and np.all(map_k_copy_0.loc[ [i[0:4] ], [ i[4:8], i[8:]]].values != 0):
              
              print('AQUI 2')
              map_k_copy.loc[ [i[0:4] ], [ i[4:8], i[8:]]] = [[1,1]]
    
    # Se resolveu todos os casos (PARE!)
    ret = case_2(map_k_copy, map_with_bits, dict_with_row_column, veio=True)

    if np.count_nonzero(ret == 1) == 0:
      return
    else:
      map_k_copy_0 = ret
      case_1(map_k_copy_0, map_with_bits)

  print(map_k_copy_0)


In [961]:
def case_4(map_k, map_with_bits, dict_with_row_column=None, veio=False):
  print("HERE - 4")

  # Lista com todas as possibilidades para agrupamento de 8
  op = possibilidades_4()

  # Copiar o mapa para fazer alterações
  map_k_copy = map_k.copy()
  map_k_copy_for_2 = map_k.copy()

  if dict_with_row_column == None:
    dict_with_row_column = {"Ag_4": {"index": [], "column": []}}
  else:
    dict_with_row_column["Ag_4"] = {"index": [], "column": []}

  for j in op:
    for i in j:
      try:
        if np.all(map_k.loc[ [i[0:4], i[4:8] ], [ i[8:12], i[12:] ]].values == [[1,1], [1,1]]):
          # print(map_k.loc[ [i[0:4], i[4:8] ], [ i[8:12], i[12:] ]].values)
        
          # Copia para pegar a simplificação para o agrupamento
          map_k_copy_0 = map_k_copy.copy()
          map_k_copy_0.loc[ [i[0:4], i[4:8] ], [ i[8:12], i[12:] ]] = [[2,2], [2,2]]
          map_k_copy_for_2.loc[ [i[0:4], i[4:8] ], [ i[8:12], i[12:] ]] = [[2,2], [2,2]]
          solve(expr(map_with_bits, map_k_copy_0, "2"), "o")
          
          # Tirando o agrupamento, ainda existe 1
          if np.count_nonzero(map_k_copy_0 == 1) >= 0:
            # print("AG de 4 try 1")
            # print(map_k_copy)

            dict_with_row_column["Ag_4"]["index"].append([i[0:4], i[4:8] ])
            dict_with_row_column["Ag_4"]["column"].append([ i[8:12], i[12:] ])
          
            map_k_copy.loc[ [i[0:4], i[4:8] ], [ i[8:12], i[12:] ]] = [[0,0], [0,0]]
          
      except:
        try:
          if np.all(map_k.loc[ [i[0:4] ], [ i[4:8], i[8:12], i[12:16], i[16:] ]].values == [[1, 1,1, 1]]):
            # print(map_k.loc[ [i[0:4] ], [ i[4:8], i[8:12], i[12:16], i[16:] ]].values)
         
            # Copia para pegar a simplificação para o agrupamento
            map_k_copy_0 = map_k_copy.copy()
            map_k_copy_0.loc[ [i[0:4] ], [ i[4:8], i[8:12], i[12:16], i[16:] ]] = [[2,2,2,2]]
            map_k_copy_for_2.loc[ [i[0:4] ], [ i[4:8], i[8:12], i[12:16], i[16:] ]] = [[2,2,2,2]]
            solve(expr(map_with_bits, map_k_copy_0, "2"), "o")
            
            # Tirando o agrupamento, ainda existe 1
            if np.count_nonzero(map_k_copy_0 == 1) >= 0:
              # print("AG de 4 try 2")
              # print(map_k_copy)

              dict_with_row_column["Ag_4"]["index"].append([i[0:4] ])
              dict_with_row_column["Ag_4"]["column"].append([ i[4:8], i[8:12], i[12:16], i[16:] ])
            
              map_k_copy.loc[ [i[0:4] ], [ i[4:8], i[8:12], i[12:16], i[16:] ]] = [[0,0,0,0]] 

        except:
          if np.all(map_k.loc[ [ i[4:8], i[8:12], i[12:16], i[16:] ], [i[0:4] ]].values == [[1], [1],[1], [1]]):
            # print(map_k.loc[ [ i[4:8], i[8:12], i[12:16], i[16:] ], [i[0:4] ]])
           
            # Copia para pegar a simplificação para o agrupamento
            map_k_copy_0 = map_k_copy.copy()
            map_k_copy_0.loc[ [ i[4:8], i[8:12], i[12:16], i[16:] ], [i[0:4] ]] = [[2], [2],[2], [2]]
            map_k_copy_for_2.loc[ [ i[4:8], i[8:12], i[12:16], i[16:] ], [i[0:4] ]] = [[2], [2],[2], [2]]
            solve(expr(map_with_bits, map_k_copy_0, "2"), "o")
            
            # Tirando o agrupamento, ainda existe 1
            if np.count_nonzero(map_k_copy_0 == 1) >= 0:
              # print("AG de 4 try 3")
              # print(map_k_copy)

              dict_with_row_column["Ag_4"]["index"].append([ i[4:8], i[8:12], i[12:16], i[16:] ])
              dict_with_row_column["Ag_4"]["column"].append([ i[0:4] ])
            
              map_k_copy.loc[ [ i[4:8], i[8:12], i[12:16], i[16:] ], [i[0:4] ]] = [[0], [0],[0], [0]]

  if np.count_nonzero(map_k_copy == 1) == 0:
    print('Resolvi no caso 4')

    if lista_fazendo != []:
      print(" + ".join(lista_fazendo))
      
      lista_fazendo.clear()
      return map_k_copy

  else:
    print("Passando pelo o 4 >>> ", lista_fazendo)
    
    # Para teste02 - AG 8
    if veio:
      return map_k_copy

    if len(dict_with_row_column["Ag_4"]['index']) > 0:
      teste03(map_k_copy, map_k_copy_for_2, map_with_bits, dict_with_row_column)
      
    else:
      case_2(map_k_copy, map_with_bits, dict_with_row_column)

In [962]:
def teste03(map_k_copy, map_k_copy_0, map_with_bits, dict_with_row_column):
  # map_test = map_k_copy_0.copy()
  print(map_k_copy_0)

  if np.count_nonzero(map_k_copy == 1) >= 1:
    print('EXISTE POSSIBILIDADES DE TER AG DE 2')
    print(map_k_copy_0)

    for j in possibilidades_2():
      for i in j:
        try:
    
            if np.any(map_k_copy_0.loc[ [i[0:4], i[4:8] ], [i[8:]]].values == 1) and np.all(map_k_copy_0.loc[ [i[0:4], i[4:8] ], [i[8:]]].values != 0):
              
              print('AQUI 1')
              # Alterando em um df diferente
              map_k_copy.loc[ [i[0:4], i[4:8] ], [i[8:]]] = [[1], [1]]

        except:
          
            if np.any(map_k_copy_0.loc[ [i[0:4] ], [ i[4:8], i[8:]]].values == 1) and np.all(map_k_copy_0.loc[ [i[0:4] ], [ i[4:8], i[8:]]].values != 0):
              
              print('AQUI 2')
              map_k_copy.loc[ [i[0:4] ], [ i[4:8], i[8:]]] = [[1,1]]
    
    # Se resolveu todos os casos (PARE!)
    ret = case_2(map_k_copy, map_with_bits, dict_with_row_column, veio=True)
    
    if np.count_nonzero(ret == 1) == 0:
      return
    else:
      map_k_copy_0 = ret
      case_1(map_k_copy_0, map_with_bits)

  print(map_k_copy_0)


In [963]:
import collections

def ver_se_resolvo_com_menas_exp(map_k, dict_with_row_column, map_with_bits):
  print("Estou dento da função que simplifica 2")
  # print(map_k)

  map_copy_reset = map_k.copy()

  index = dict_with_row_column["Ag_2"]["index"]
  column = dict_with_row_column["Ag_2"]["column"]
  
  for i,j in zip(index, column):
    
    if collections.Counter(map_k.loc[ i, j].values.reshape(-1,1)[...,0]) == collections.Counter([1,1]) or collections.Counter(map_k.loc[ i, j].values.reshape(-1,1)[...,0]) == collections.Counter([1,2]):

      map_copy_0 = map_copy_reset.copy()

      map_k.loc[ i, j] = 2
      
      map_copy_0.loc[ i, j] = 2
      
      solve(expr(map_with_bits, map_copy_0, "2"), "o")

In [964]:
def case_2(map_k, map_with_bits, dict_with_row_column=None, veio=False):
  print("HERE - 2")
  # Lista com todas as possibilidades para agrupamento de 8
  op = possibilidades_2()

  # Salvando para se eu modificar, não perder o conteudo anterior dos outros agrupamentos
  lista_fazendo_original = lista_fazendo.copy()
  
  # Copiar o mapa para fazer alterações
  map_k_copy = map_k.copy()

  if dict_with_row_column == None:
    dict_with_row_column = {"Ag_2": {"index": [], "column": []}}
  else:
    dict_with_row_column["Ag_2"] = {"index": [], "column": []}
  
  for j in op:
    for i in j:
      try:
        
        if list(map_k.loc[ [i[0:4], i[4:8] ], [i[8:]]].values) == [[1], [1]]:
          # print(map_k.loc[ [i[0:4], i[4:8] ], [i[8:]]]) 

          # Copia para pegar a simplificação para o agrupamento
          map_k_copy_0 = map_k_copy.copy()
          map_k_copy_0.loc[ [i[0:4], i[4:8] ], [i[8:]]] = [[2], [2]]
          solve(expr(map_with_bits, map_k_copy_0, "2"), "o")
          
          # Tirando o agrupamento, ainda existe 1
          if np.count_nonzero(map_k_copy_0 == 1) >= 0:
            # print("AG de 2 try 1")
            # print(map_k_copy)

            dict_with_row_column["Ag_2"]["index"].append([i[0:4], i[4:8] ])
            dict_with_row_column["Ag_2"]["column"].append([i[8:]])
          
            map_k_copy.loc[ [i[0:4], i[4:8] ], [i[8:]]] = [[0], [0]]
            

            # Isso serve para o teste3
            if veio:
              map_k.loc[ [i[0:4], i[4:8] ], [i[8:]]] = [[0], [0]]

      except:

        if np.all(map_k.loc[ [i[0:4] ], [ i[4:8], i[8:]]].values == [[1,1]]):
          # print(map_k.loc[ [i[0:4] ], [ i[4:8], i[8:]]])
          
          # Copia para pegar a simplificação para o agrupamento
          map_k_copy_0 = map_k_copy.copy()
          map_k_copy_0.loc[ [i[0:4] ], [ i[4:8], i[8:]]] = [[2, 2]]
          solve(expr(map_with_bits, map_k_copy_0, "2"), "o")
          
          # Tirando o agrupamento, ainda existe 1
          if np.count_nonzero(map_k_copy_0 == 1) >= 0:
            # print("AG de 2 try 2")
            # print(map_k_copy)

            dict_with_row_column["Ag_2"]["index"].append([i[0:4] ])
            dict_with_row_column["Ag_2"]["column"].append([ i[4:8], i[8:]])
          
            map_k_copy.loc[ [i[0:4] ], [ i[4:8], i[8:]]] = [[0,0]]

            
            # Isso serve para o teste3
            if veio:
              map_k.loc[ [i[0:4] ], [ i[4:8], i[8:]]] = [[0,0]]
  
  # lista_fazendo.clear()
  
  # Esse reduz mais ainda a expressão de agrupamento de 2
  # ver_se_resolvo_com_menas_exp(map_k, dict_with_row_column, map_with_bits)

  # if len(lista_fazendo_original) > 0:
  #   print("Voltando os valores para a lista")
  #   print(lista_fazendo)
  #   for i in lista_fazendo_original:
  #     lista_fazendo.append(i)

  if np.count_nonzero(map_k_copy == 1) == 0:
    print('Resolvi - Case 2')

    if lista_fazendo != []:
      print("x ="," + ".join(lista_fazendo))
      lista_fazendo.clear()
      return map_k_copy
    else:
      print("Lista_original")
      print("x ="," + ".join(lista_fazendo_original))
      
    return map_k_copy

  else:
    print("Passando pelo o 2 >>> ", lista_fazendo)
    
    if veio:
      return map_k_copy
    else:
      # print(map_k_copy)
      case_1(map_k_copy, map_with_bits)

In [965]:
def case_1(map_k, map_with_bits):
  print("HERE - 1")
  print(f" ----------------- Expr_2 ----------------- {1}")
  
  if np.count_nonzero(map_k == 1) > 1:
    print('Resolvi - Case 1 com mais de 1 isolado')
    print("x = ", " + ".join(expr(map_with_bits, map_k, "1")))
  
  else:
    print('Resolvi - Case 1')
    solve(expr(map_with_bits, map_k, "1"), "o")
    print(lista_fazendo)
    
    print("Parei aqui >>> x =", " + ".join(lista_fazendo))

  lista_fazendo.clear()   

In [967]:
# MAIN
x = [ 
      0,0, 0,1,

      0,0, 0,0,
      0,0, 0,0,
      0,0, 0,0,
    ]
lista_fazendo.clear()
map_ = map_k(x)
simplifier(map_[0], map_[1])

map_[0]

HERE - 1
 ----------------- Expr_2 ----------------- 1
Resolvi - Case 1
["A'B'C D "]
Parei aqui >>> x = A'B'C D 


,C'D',C'D,C D,C D'
A'B',0,0,1,0
A'B,0,0,0,0
A B,0,0,0,0
A B',0,0,0,0


In [954]:
map_[1]

,C'D',C'D,C D,C D'
A'B',1 - 0000,1 - 0001,1 - 0011,1 - 0010
A'B,1 - 0100,1 - 0101,1 - 0111,1 - 0110
A B,0 - 1100,0 - 1101,1 - 1111,0 - 1110
A B',1 - 1000,1 - 1001,1 - 1011,1 - 1010


In [955]:
def simplifier(map_k, map_with_bits):
  # Case 16
  if np.all(np.array(map_k).reshape(1,-1)[0] == np.ones(np.array(map_k).size, dtype=int)):
    return 1
  
  # Case 8
  if np.count_nonzero(map_k == 1) >= 8:
    return case_8(map_k, map_with_bits)
  
  # Case 4
  if np.count_nonzero(map_k == 1) >= 4:
    return case_4(map_k, map_with_bits)
   
  # Case 2
  if np.count_nonzero(map_k == 1) >= 2:
    return case_2(map_k, map_with_bits)

  # Case 1
  if np.count_nonzero(map_k == 1) >= 1:
    return case_1(map_k, map_with_bits)

In [956]:
# Descartado

list_with_bits = []
def expr_0(map_with_bits, map_k, s):
  print(f" ---- Expr_2 - list_with_bits ---- {s}")
  expression = ""

  for i in range(np.count_nonzero(map_k == 2)):
    expression += map_with_bits.loc[ 
                                          [list(map_k.iloc[np.where(map_k == 2)].index)[i]],
                                          [list(map_k.iloc[np.where(map_k == 2)].columns)[i]] 
                          ].values[0][0][4:]
    expression += " "

  list_with_bits.append(expression)


In [957]:
# Anotação

A list_with_bits retorna uma lista com os bits para um agrupamento

Exemplo:
  
  ['0000 0001 0011 0010 0100 0101 0111 0110 ']

SyntaxError: invalid syntax (2561898810.py, line 3)

In [ ]:
# Lembre-se que o conjunto ele embaralha os dados
  mask = "".join([
                  "".join(list(set(map_k_copy.iloc[ np.where(map_k_copy == 1) ].index))),
                  "".join(list(map_k_copy.iloc[ np.where(map_k_copy == 1) ].columns))
                ]) 

  lista_com_possibilidades = []
  print("MASCARA: ",mask)
  
  ## Pronto ta encontrando já. Falta fazer a logica pra mudar os bits
  
  if np.count_nonzero(map_k_copy_0 == 1) == 2:
    for i in possibilidades_4():
      print(i)
      for j in i:
        if mask in j:
          lista_com_possibilidades.append(j)
  print(map_k_copy_0)